In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from scipy import stats
import warnings
from scipy.special import boxcox1p
from scipy.stats import norm, skew
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

In [69]:
train_external= pd.read_csv('train_updated.csv', index_col=0)
test_external = pd.read_csv('test_updated.csv', index_col=0)

In [102]:
df_test_original = pd.read_csv('test.csv')
df_train_original = pd.read_csv('train.csv')

y_train=df_train_original['resale_price']/100000

df_train_original=df_train_original.drop(columns='resale_price')

## Adding External Factors 

11 new features from auxiliary data and external data are added. Some features including primary_1, primary_2, hawker_1, shopping_1, commercial_1 are without date info and not accurate. These inaccurate features will be validated in the feature selection and further explained in the limitations.

In [71]:
def add_feature(orginal, external): 
    add_columns=['commercial_1','mrt_0_5','mrt_1_0','mrt_1_5','hawker_1',
             'primary_1','primary_2','mrt_closest','distance_to_orchard',
             'shopping_mall_1','gold_price']
    
    df= pd.concat([orginal, external[add_columns]],axis=1, join='inner')
    return df 

df_train_added=add_feature(df_train_original,train_external)
df_test_added=add_feature(df_test_original,test_external)

In [72]:
def preprocesor(df):
#assuption a: For month, it is a string format and it will be replaced with two new numerical features, namely ‘saleyear’ and ‘salemonth’
    df['sale_year'] = df['month'].apply(lambda x: x.split("-")[0]).astype('int')
    df['sale_month'] = df['month'].apply(lambda x: x.split("-")[1]).astype('int') 
#assuption b: room_number and toilet number are created     
    df['flat_type'] = df['flat_type'].apply(lambda x : x.replace('-', ' ').lower())
    df['room_number'] = df['flat_type'].map({'2 room': 2,'3 room': 3,'4 room': 4,'5 room': 5,
                            'executive': 3, 'multi generation': 4, '1 room': 1})
    df['toilet_number'] = df['flat_type'].map({ '1 room': 1, '2 room': 1, '3 room': 2,'4 room': 2, '5 room': 2,
        'executive': 2, 'multi generation': 3})

    
#assumption e: With str format, two features structured as  storeylow and storeyhigh with int format replace 
    df['storey_low'] = df['storey_range'].apply(lambda x: x.split(" ")[0]).astype('int')
    df['storey_high'] = df['storey_range'].apply(lambda x: x.split(" ")[2]).astype('int')

#assumption f: lease_commerce_year is created 

    df['lease_commence_year']= df['sale_year']-df['lease_commence_date']
#assumption c,d,g: remove columns 
    
    removal=['month','block','street_name','eco_category','elevation','storey_range','flat_type','lease_commence_date']
    
    removal_geo=['latitude','longitude','town','planning_area','region']
    
    df = df.drop(columns=removal,axis=1)
    df = df.drop(columns=removal_geo,axis=1)
    df = pd.get_dummies(df)
    print(df.shape)  
    return df

In [16]:
class SklearnModelWithName:
    def __init__(self,model_name,model_definition):
        self.model_name = model_name
        self.model_definition = model_definition

In [17]:
elasticnet = make_pipeline(RobustScaler(), ElasticNet(alpha =0.00001, random_state=1))
elasticnet = SklearnModelWithName("elasticnet",elasticnet)

svm = make_pipeline(RobustScaler(),SVR(C=140))
svm  = SklearnModelWithName("svm",svm)


lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
lasso = SklearnModelWithName("lasso",lasso)

GBoost = GradientBoostingRegressor(loss='huber',min_samples_split=10,n_estimators=200)
GBoost = SklearnModelWithName("gboost",GBoost)

extra_tree = ExtraTreesRegressor(n_estimators=100,n_jobs=4,min_samples_split=10)
extra_tree = SklearnModelWithName("extra_tree",extra_tree)

random_forest = RandomForestRegressor(n_estimators=100,n_jobs=4,min_samples_split=10)
random_forest = SklearnModelWithName("random_forest",random_forest)

krr = KernelRidge(alpha=1e-5,kernel='polynomial',degree=3)
krr = SklearnModelWithName("krr",krr)

class StackClassifer(BaseEstimator, TransformerMixin, RegressorMixin):
    def __init__(self,base_models,meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit_model(self,_model,_x,_y):
        _model.fit(_x,_y)
        return _model
        
    def fit(self,X,y):
        self.base_models_ = clone(self.base_models)
        self.meta_model_ = clone(self.meta_model)
        for base_model in self.base_models_:
            base_model.fit(X,y)
        y_pred = np.array([base_model.predict(X) for base_model in self.base_models_])
        meta_train = y_pred.mean(axis=0,keepdims=True).T
        self.meta_model_.fit(meta_train,y)
        return self
    
    def predict(self,X):
        y_pred = np.array([base_model.predict(X) for base_model in self.base_models_])
        meta_train = y_pred.mean(axis=0,keepdims=True).T
        return self.meta_model_.predict(meta_train)

In [73]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [103]:
X_train=preprocesor(df_train_added)
X_test=preprocesor(df_test_added)

(431732, 194)
(107934, 194)


In [104]:
import time
start= time.time()
X=X_train
Y=y_train

random_forest = make_pipeline(RobustScaler(), RandomForestRegressor(n_estimators=150,n_jobs=4,min_samples_split=10))
random_forest = SklearnModelWithName("random_forest",random_forest)

extra_tree = make_pipeline(RobustScaler(),ExtraTreesRegressor(n_estimators=100,n_jobs=4,min_samples_split=10))
extra_tree = SklearnModelWithName("extra_tree",extra_tree)

mlp = make_pipeline(RobustScaler(), MLPRegressor(hidden_layer_sizes=(80, 80, 50), alpha=0.00001, random_state=1))
mlp = SklearnModelWithName("mlp",mlp)

base_models_w_name = [mlp, extra_tree, random_forest]
meta_model_w_name = lasso
base_models = [o.model_definition for o in base_models_w_name]
meta_model = meta_model_w_name.model_definition

stack_clf = StackClassifer(base_models,meta_model)
stack_clf.fit(X,Y)

end= time.time()

print('training_time',end-start)

training_time 820.10759806633


In [105]:
y_pred = stack_clf.predict(X)

print(rmsle(y_train,y_pred))

0.01164058532874655


In [88]:
y_test=stack_clf.predict(X_test)

In [89]:
y_test=100000*y_test

In [98]:
submission_df = pd.DataFrame(y_test)

In [99]:
submission_df['Id'] = submission_df.index

In [100]:
submission_df['Id'] = submission_df.index
submission_df['Predicted'] = submission_df[0]
del submission_df[0]
submission_df.to_csv('submissionapr17_100000.csv', index=False)

In [ ]:
import time
start= time.time()
X=X_train
Y=y_train_10

random_forest = make_pipeline(RobustScaler(), RandomForestRegressor(n_estimators=150,n_jobs=4,min_samples_split=10))
random_forest = SklearnModelWithName("random_forest",random_forest)

extra_tree = make_pipeline(RobustScaler(),ExtraTreesRegressor(n_estimators=100,n_jobs=4,min_samples_split=10))
extra_tree = SklearnModelWithName("extra_tree",extra_tree)

mlp = make_pipeline(RobustScaler(), MLPRegressor(hidden_layer_sizes=(80, 80, 50), alpha=0.00001, random_state=1))
mlp = SklearnModelWithName("mlp",mlp)

base_models_w_name = [mlp, extra_tree, random_forest]
meta_model_w_name = lasso
base_models = [o.model_definition for o in base_models_w_name]
meta_model = meta_model_w_name.model_definition

stack_clf = StackClassifer(base_models,meta_model)
stack_clf.fit(X,Y)

end= time.time()

print('training_time',end-start)